In [1]:
#pip install pyreadstat
"""

- This file is used to create 
    1) Population denominator table (df_denom, aka "Demographic_by_registerpop_year_dec22_v1.xlsx")
    2) Disease by register by year
        a) N-table (df_results "Demographic_by_disease_year_dec22_v 2/3 .xlsx")
        b) Rate-table
        
- this script aim to be a cleaner version of previos (Table_Dis_by_register_year_dec22v3)

""" 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pyreadstat
import patsy
import pyreadstat
import statsmodels.formula.api as smf
import datetime as dt
from datetime import datetime
import time # for sleep
%matplotlib inline

In [287]:
#
dtchangedic = {"LopNr":'int32',
               'FoddAr':'int32',
           "FodelseLandG":'category',
           "DispInkFamLag5_1":'float32',
           "DispInkFamLag5_1_r":'float32',
           'Lan':'category', 
           #"Civil":'category',
           "Sun2000niva":'int32',
           "PC_u":'float32',
           "PC_e":'float32',
        "SO_u":'float32',
          "SO_e":'float32',
           "SI_u":'float32',
          "SI_e":'float32',
          'Death':'float32'}

In [3]:
def calculate_t(n,s):
    """
    Program for calculating täljare used for pooled varaince
    """
    t = (n-1)*(s**2)
    return t

In [205]:
def chiCreator(n_Q1_x,n_Q1_not_x, n_Q5_x, n_Q5_not_x ): 
    """
    In: 
        - n_Q1_x
        - n_Q1_not_x 
        - n_Q5_x 
        - n_Q5_not
    Out: 
        -ch2resut2print: text for chi2 
    """
    contingency_table = pd.DataFrame({'Q1': [n_Q1_x, n_Q1_not_x],
                                     'Q5': [n_Q5_x, n_Q5_not_x]})

    chi2, p_value_chi2, _, _= stats.chi2_contingency(contingency_table)
    if p_value_chi2 < 0.001:
        p2print = 'p<.001'
    else:
        p2print =f"{p_value_chi2:.2f}"
    ch2resut2print = f"\u03C7\u00b2={chi2:.2f}, {p2print}"
    return ch2resut2print

In [228]:
def rateBinary(married, unmarried):
    
    r = (married/(married + unmarried))*100
    r2print =  f"{r:.2f}%"
    return r2print

In [216]:
def rateQ5Q12(Q5ink, Q1ink):
    r = Q5ink/(Q1ink)
    r2print =  f"{r:.2f}"
    return r2print

In [230]:
%%time

"""
- löp över disease, alla år, ladda data korrekt. 
-Sort relevant data (same as for fig 1)
- Loop to create 
    -df_results
    -df_denom
- 
"""
        # Remove comment for below,and remove me
        # #del df, meta


SESvarName ="DispInkFamLag5_1_r"
SESvarNamePrecursor = 'DispInkFamLag5_1_r'
SESlow = 0#"1"#'A' # A =rural, B eller C =centralort
SESmid = 2
SEShigh =4#"2" #'C'



startYear= 2004
endYear= 2018 #exlusive end
print(startYear)

gbdNr2namedic ={8:"Neoplasm",9:'Cardiovascular',19:'Musculoskeletal',13:'Mental',12:'Neurological' , 
                10:'Chronic_respiratory' , 15: 'Diabetes', 23: 'Self-harm_violence',100: 'All'}

regList = ['PC_u', 'SO_u', 'SI_u', 'Death']

df_results=pd.DataFrame()
df_denom = pd.DataFrame()
df_pop = pd.DataFrame()
rowN =0
rowA = 1
rowAstd = 2



rowNpopd = 0  
rowApopd = 1 
rowAstdpopd = 2 
rowMRpopd = 3   
rowMChipopd = 4   
rowSweRpopd = 5 
rowSweChipopd = 6  
rowQ5bQ1popd = 7  
                


#make disease specific df
for DisNr in [100,9, 8 , 12,10,15]: #[9, 8 , 12, 13,19]
    for AR in range(startYear,endYear):#2020)

        start = time.time()

        #load data
        fp = 'C:X\gbddis_%s_%s.sas7bdat' %(DisNr, AR)
        
        df, meta = pyreadstat.read_sas7bdat(fp, encoding="LATIN1")
        #send_ping(DisNr,AR,add = 'loaded first')

        print('TRACK: DisNr ', DisNr,' for ',AR,' was loaded, //T:',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        
        # CLEAN data frame from rows with missing values of essential variables
        df = df[df[SESvarNamePrecursor].notna()] # keep only valid 

        
        
        #df = df[df.Desoletter.isin(['A', 'B', 'C'])]
        df = df[df.LopNr>0] # only lopnr largrer than 0 (some counties provided wrong lopnr)
        df = df[df[SESvarName].notna()] # only include rows where we have proper income rank
        df = df[df.Kon.isin(["1","2"])]
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df = df.fillna(0)
        
        df['Sun2000niva'] = pd.to_numeric(df['Sun2000niva'],errors='coerce') # creates nan for invalid edu
        df = df[df["Sun2000niva"].notna()] # remove invalid edu
        
        #only include proper Län /needed for regression
        df = df[df.Lan.isin(['01','03','04','05','06','07','08','09','10','12','13','14','17','18','19','20','21','22','23','24','25'])]
        
        df['FoddAr'] = pd.to_numeric(df['FoddAr'],errors='coerce') # creates nan for invalid FoddAr
        df = df[df["FoddAr"].notna()] # removes rows with missing FoddAr
        
        # for LISA > 2010, also 15 yrs old are included. Make konsekvent, remove all 15 and below. 
        df['Age'] = AR-df['FoddAr']
        df = df[df['Age']>15]
        df['AgeP2'] = df['Age']**2     # non-linear age effects
        df['CivStat'] = 0              # default Civilst ej gift
        df.loc[df['Civil'] == 'G','CivStat' ] = 1 # 1 if gift. 
        df.drop(columns="Civil", inplace=True)
        df = df.astype(dtchangedic, copy=False, errors='raise') #dtchangedic specified above // copy=True,
        
        # fixa PC_U_a  
        df['PC_U_a']= df['PC_u'] - df['SO_u'] 
        df.loc[df['PC_U_a']<0, "PC_U_a"]= 0  #remove neg


        # create PC denominator
        ar = AR
        df['PC_denom'] = df['Lan'].apply(lambda i: 1 if (          (i =='01' and ar>= 2004) or  
                                                                   (i =='03' and ar>= 2010) or
                                                                   (i =='05' and ar>= 1999)  or
                                                                   (i =='06' and ar>= 2012)  or
                                                                   (i =='08' and ar>= 2011)  or
                                                                   (i =='09' and ar>= 2013)  or
                                                                   (i =='10' and ar>= 2010)  or
                                                                   (i =='12' and ar>= 2004)  or
                                                                   (i =='14' and ar>= 2000)  or
                                                                   (i =='17' and ar>= 2014)  or
                                                                   (i =='18' and ar>= 2002)  or
                                                                   (i =='19' and ar>= 2017)  or
                                                                   (i =='20' and ar>= 2005)  or
                                                                   (i =='21' and ar>= 2010)  or
                                                                   (i =='22' and ar>= 2011)  or
                                                                   (i =='23' and ar>= 2010)  or
                                                                   (i =='25' and ar>= 2012)
                                                                    )                                         
                                                        else 0)  

        df['NPR_denom'] = 1

        df['year'] = AR
        
        DisName = gbdNr2namedic[DisNr]
            # innermost loop
        for reg in regList:
            koko =0

        # outside reg loop, create population denominator df (df_denom)
        # for making df_denom of all included (also make df_pop, for all incl not included) 
        if DisNr == 100:# Skip since done 100: #only do df_denom once
            for pops in ['NPR_denom', 'PC_denom']:
                df_denom.loc[rowNpopd,'dis_n_denom'] = 'TotPop'
                df_denom.loc[rowNpopd,'year'] = AR
                df_denom.loc[rowNpopd,'reg'] = pops
                df_denom.loc[rowNpopd,'measure'] = 'N'
                df_denom.loc[rowNpopd,'All_mf'] =int(df.loc[(df.Kon.isin(['1','2'])) ,pops].sum())
                df_denom.loc[rowNpopd,'All_m'] = int(df.loc[(df.Kon.isin(['1'])) ,pops].sum())
                df_denom.loc[rowNpopd,'All_f'] = int(df.loc[(df.Kon.isin(['2'])) ,pops].sum())
                df_denom.loc[rowNpopd,'Q0_mf'] = int(df.loc[(df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0),pops].sum())
                df_denom.loc[rowNpopd,'Q0_m'] =  int(df.loc[(df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==0),pops].sum())
                df_denom.loc[rowNpopd,'Q0_f'] =  int(df.loc[(df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==0),pops].sum())
                df_denom.loc[rowNpopd,'Q2_mf'] = int(df.loc[(df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==2),pops].sum())
                df_denom.loc[rowNpopd,'Q2_m'] =  int(df.loc[(df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==2),pops].sum())
                df_denom.loc[rowNpopd,'Q2_f'] =  int(df.loc[(df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==2),pops].sum())
                df_denom.loc[rowNpopd,'Q4_mf'] = int(df.loc[(df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4),pops].sum())
                df_denom.loc[rowNpopd,'Q4_m'] =  int(df.loc[(df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==4),pops].sum())
                df_denom.loc[rowNpopd,'Q4_f'] =  int(df.loc[(df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==4),pops].sum())


                df_denom.loc[rowApopd,'dis_n_denom'] = 'TotPop'
                df_denom.loc[rowApopd,'year'] = AR
                df_denom.loc[rowApopd,'reg'] = pops
                df_denom.loc[rowApopd,'measure'] = 'MeanAge'
                df_denom.loc[rowApopd,'All_mf'] =df.loc[((df.Kon.isin(['1','2'])) & (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'All_m'] = df.loc[((df.Kon.isin(['1'])) & (df[pops]==1)) ,'Age'].mean()
                df_denom.loc[rowApopd,'All_f'] = df.loc[((df.Kon.isin(['2'])) & (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'Q0_mf'] = df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'Q0_m'] =  df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==0)& (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'Q0_f'] =  df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==0)& (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'Q2_mf'] = df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==2) & (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'Q2_m'] =  df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==2) & (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'Q2_f'] =  df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==2) & (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'Q4_mf'] = df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1)),'Age'].mean()
                df_denom.loc[rowApopd,'Q4_m'] =  df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1)) ,'Age'].mean()
                df_denom.loc[rowApopd,'Q4_f'] =  df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1)),'Age'].mean()
                                                         
                df_denom.loc[rowAstdpopd,'dis_n_denom'] = 'TotPop'
                df_denom.loc[rowAstdpopd,'year'] = AR
                df_denom.loc[rowAstdpopd,'reg'] = pops
                df_denom.loc[rowAstdpopd,'measure'] = 'AgeStd'
                df_denom.loc[rowAstdpopd,'All_mf'] =df.loc[((df.Kon.isin(['1','2'])) & (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'All_m'] = df.loc[((df.Kon.isin(['1'])) & (df[pops]==1)) ,'Age'].std()
                df_denom.loc[rowAstdpopd,'All_f'] = df.loc[((df.Kon.isin(['2'])) & (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'Q0_mf'] = df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'Q0_m'] =  df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==0)& (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'Q0_f'] =  df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==0)& (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'Q2_mf'] = df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==2) & (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'Q2_m'] =  df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==2) & (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'Q2_f'] =  df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==2) & (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'Q4_mf'] = df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1)),'Age'].std()
                df_denom.loc[rowAstdpopd,'Q4_m'] =  df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1)) ,'Age'].std()
                df_denom.loc[rowAstdpopd,'Q4_f'] =  df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1)),'Age'].std()
                
                
                # rate married
                df_denom.loc[rowMRpopd,'dis_n_denom'] = 'TotPop'
                df_denom.loc[rowMRpopd,'year'] = AR
                df_denom.loc[rowMRpopd,'reg'] = pops
                df_denom.loc[rowMRpopd,'measure'] = 'married'
                df_denom.loc[rowMRpopd,'All_mf'] = rateBinary(df.loc[((df.Kon.isin(['1','2'])) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                 df.loc[((df.Kon.isin(['1','2'])) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                df_denom.loc[rowMRpopd,'All_m'] =  rateBinary(df.loc[((df.Kon.isin(['1'])) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                 df.loc[((df.Kon.isin(['1'])) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                df_denom.loc[rowMRpopd,'All_f'] =  rateBinary(df.loc[((df.Kon.isin(['2'])) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                 df.loc[((df.Kon.isin(['2'])) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                
                df_denom.loc[rowMRpopd,'Q0_mf'] = rateBinary(df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                 df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                df_denom.loc[rowMRpopd,'Q0_m'] = rateBinary(df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                df_denom.loc[rowMRpopd,'Q0_f'] = rateBinary(df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                df_denom.loc[rowMRpopd,'Q2_mf'] = np.nan 
                df_denom.loc[rowMRpopd,'Q2_m']  = np.nan
                df_denom.loc[rowMRpopd,'Q2_f']  = np.nan 
                df_denom.loc[rowMRpopd,'Q4_mf'] = rateBinary(df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                 df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                df_denom.loc[rowMRpopd,'Q4_m'] = rateBinary(df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==4)  & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                df_denom.loc[rowMRpopd,'Q4_f'] = rateBinary(df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==4)  & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                
                # chi2 married
                df_denom.loc[rowMChipopd,'dis_n_denom'] = 'TotPop'
                df_denom.loc[rowMChipopd,'year'] = AR
                df_denom.loc[rowMChipopd,'reg'] = pops
                df_denom.loc[rowMChipopd,'measure'] = 'chi2_married'
                df_denom.loc[rowMChipopd,'All_mf'] = chiCreator(df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(),
                                                                df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum(), 
                                                                df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.CivStat == 1 ) ), pops].sum(), 
                                                                df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.CivStat != 1 ) ), pops].sum())
                df_denom.loc[rowMChipopd,'Q0_mf']= np.nan
                df_denom.loc[rowMChipopd,'Q0_m'] = np.nan 
                df_denom.loc[rowMChipopd,'Q0_f'] = np.nan  
                df_denom.loc[rowMChipopd,'Q2_mf']= np.nan 
                df_denom.loc[rowMChipopd,'Q2_m'] = np.nan
                df_denom.loc[rowMChipopd,'Q2_f'] = np.nan 
                df_denom.loc[rowMChipopd,'Q4_mf']= np.nan
                df_denom.loc[rowMChipopd,'Q4_m'] = np.nan 
                df_denom.loc[rowMChipopd,'Q4_f'] = np.nan 
                
                # rate Swe-born
                df_denom.loc[rowSweRpopd,'dis_n_denom'] = 'TotPop'
                df_denom.loc[rowSweRpopd,'year'] = AR
                df_denom.loc[rowSweRpopd,'reg'] = pops
                df_denom.loc[rowSweRpopd,'measure'] = 'SweBorn'
                df_denom.loc[rowSweRpopd,'All_mf'] = rateBinary(df.loc[((df.Kon.isin(['1','2'])) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                                 df.loc[((df.Kon.isin(['1','2'])) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                df_denom.loc[rowSweRpopd,'All_m'] =  rateBinary(df.loc[((df.Kon.isin(['1'])) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                                 df.loc[((df.Kon.isin(['1'])) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                df_denom.loc[rowSweRpopd,'All_f'] =  rateBinary(df.loc[((df.Kon.isin(['2'])) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                                 df.loc[((df.Kon.isin(['2'])) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                
                df_denom.loc[rowSweRpopd,'Q0_mf'] = rateBinary(df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                               df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                df_denom.loc[rowSweRpopd,'Q0_m'] =  rateBinary(df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                               df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                df_denom.loc[rowSweRpopd,'Q0_f'] =  rateBinary(df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                               df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                df_denom.loc[rowSweRpopd,'Q2_mf'] = np.nan 
                df_denom.loc[rowSweRpopd,'Q2_m']  = np.nan
                df_denom.loc[rowSweRpopd,'Q2_f']  = np.nan 
                df_denom.loc[rowSweRpopd,'Q4_mf'] = rateBinary(df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                               df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                df_denom.loc[rowSweRpopd,'Q4_m']  = rateBinary(df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==4)     & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                               df.loc[((df.Kon.isin(['1'])) & (df['DispInkFamLag5_1_r']==4)     & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                df_denom.loc[rowSweRpopd,'Q4_f']  = rateBinary(df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==4)     & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                               df.loc[((df.Kon.isin(['2'])) & (df['DispInkFamLag5_1_r']==4)     & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                
                # chi2 Swe-born
                df_denom.loc[rowSweChipopd,'dis_n_denom'] = 'TotPop'
                df_denom.loc[rowSweChipopd,'year'] = AR
                df_denom.loc[rowSweChipopd,'reg'] = pops
                df_denom.loc[rowSweChipopd,'measure'] = 'chi2_Swe'
                df_denom.loc[rowSweChipopd,'All_mf'] = chiCreator(df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(),
                                                                df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum(), 
                                                                df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.FodelseLandG == 1 ) ), pops].sum(), 
                                                                df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1) & (df.FodelseLandG != 1 ) ), pops].sum())
                df_denom.loc[rowSweChipopd,'Q0_mf']= np.nan
                df_denom.loc[rowSweChipopd,'Q0_m'] = np.nan 
                df_denom.loc[rowSweChipopd,'Q0_f'] = np.nan  
                df_denom.loc[rowSweChipopd,'Q2_mf']= np.nan 
                df_denom.loc[rowSweChipopd,'Q2_m'] = np.nan
                df_denom.loc[rowSweChipopd,'Q2_f'] = np.nan 
                df_denom.loc[rowSweChipopd,'Q4_mf']= np.nan
                df_denom.loc[rowSweChipopd,'Q4_m'] = np.nan 
                df_denom.loc[rowSweChipopd,'Q4_f'] = np.nan 
                
                
                
                # Q5/Q1
                df_denom.loc[rowQ5bQ1popd,'dis_n_denom'] = 'TotPop'
                df_denom.loc[rowQ5bQ1popd,'year'] = AR
                df_denom.loc[rowQ5bQ1popd,'reg'] = pops
                df_denom.loc[rowQ5bQ1popd,'measure'] = 'ink_Q5_5_1byQ1_5_1'
                df_denom.loc[rowQ5bQ1popd,'All_mf'] = rateQ5Q12(df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==4) & (df[pops]==1)  ), "DispInkFamLag5_1"].sum(),
                                                                df.loc[((df.Kon.isin(['1','2'])) & (df['DispInkFamLag5_1_r']==0) & (df[pops]==1)  ), "DispInkFamLag5_1"].sum()) 

                df_denom.loc[rowQ5bQ1popd,'Q0_mf']= np.nan
                df_denom.loc[rowQ5bQ1popd,'Q0_m'] = np.nan 
                df_denom.loc[rowQ5bQ1popd,'Q0_f'] = np.nan  
                df_denom.loc[rowQ5bQ1popd,'Q2_mf']= np.nan 
                df_denom.loc[rowQ5bQ1popd,'Q2_m'] = np.nan
                df_denom.loc[rowQ5bQ1popd,'Q2_f'] = np.nan 
                df_denom.loc[rowQ5bQ1popd,'Q4_mf']= np.nan
                df_denom.loc[rowQ5bQ1popd,'Q4_m'] = np.nan 
                df_denom.loc[rowQ5bQ1popd,'Q4_f'] = np.nan 
                

 
                rowNpopd = rowNpopd + 8
                rowApopd = rowApopd + 8
                rowAstdpopd = rowAstdpopd + 8
                rowMRpopd = rowMRpopd + 8 
                rowMChipopd = rowMChipopd + 8
                rowSweRpopd = rowSweRpopd + 8
                rowSweChipopd = rowSweChipopd + 8
                rowQ5bQ1popd = rowQ5bQ1popd + 8
                
                
        # Remove comment for below
        del df, meta 
        
        excelName2save_df_results = "Demographic_by_disease_year_june23.xlsx"  #Demographic_by_disease_year_dec22_v2
        #df_results.to_excel(excelName2save_df_results)

        excelName2save_df_denoms = "Demographic_by_registerpop_year_june23.xlsx"  #empty, why? Demographic_by_registerpop_year_dec22_v1.xlsx
        df_denom.to_excel(excelName2save_df_denoms)



2004
TRACK: DisNr  100  for  2004  was loaded, //T: 05/06/2023 18:24:26
TRACK: DisNr  100  for  2005  was loaded, //T: 05/06/2023 18:26:26
TRACK: DisNr  100  for  2006  was loaded, //T: 05/06/2023 18:28:13
TRACK: DisNr  100  for  2007  was loaded, //T: 05/06/2023 18:30:00
TRACK: DisNr  100  for  2008  was loaded, //T: 05/06/2023 18:35:55
TRACK: DisNr  100  for  2009  was loaded, //T: 05/06/2023 18:42:01
TRACK: DisNr  100  for  2010  was loaded, //T: 05/06/2023 18:47:52
TRACK: DisNr  100  for  2011  was loaded, //T: 05/06/2023 18:53:36
TRACK: DisNr  100  for  2012  was loaded, //T: 05/06/2023 18:59:53
TRACK: DisNr  100  for  2013  was loaded, //T: 05/06/2023 19:05:36
TRACK: DisNr  100  for  2014  was loaded, //T: 05/06/2023 19:11:06
TRACK: DisNr  100  for  2015  was loaded, //T: 05/06/2023 19:16:55
TRACK: DisNr  100  for  2016  was loaded, //T: 05/06/2023 19:22:37
TRACK: DisNr  100  for  2017  was loaded, //T: 05/06/2023 19:28:26
Wall time: 1h 6min 2s


In [67]:
# if not building df_results from scratch, load here. 
df_results = pd.read_excel("Demographic_by_disease_year_feb23.xlsx")
df_results.drop(columns='Unnamed: 0', inplace=True) # clean

# For disease by reg by year, numbers

In [68]:
# For tot pop 

# create df "ll" with mulitindex, and crate "Full period"- values

reg_or_den_List = ['PC_u', 'SO_u', 'SI_u', 'Death', 'NPR_denom', 'PC_denom']
dis_n_denom_List = ['All', 'Cardiovascular', 'Neoplasm', 'Neurological','Diabetes','Chronic_respiratory', 'TotPop']
demog_col_List =["All_mf","All_m","All_f","Q0_mf","Q0_m","Q0_f","Q2_mf","Q2_m","Q2_f", "Q4_mf","Q4_m","Q4_f"]

#df_denom_resuls = pd.concat([df_results, df_denom])
ll = df_results.groupby(['dis_n_denom','reg', 'year','measure' ]).sum()
for nd, dis_or_tot in enumerate(dis_n_denom_List[:-1]):#, 'Cardiovascular']):#dis_or_tot_List[:-1]): 
    for dn, reg_or_den in enumerate(reg_or_den_List[:-2]) :
        
        ll_n = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'N'),:].copy()
        ll_n_u = ll_n.unstack()
        ll_a = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'MeanAge'),:].copy()
        ll_a_u = ll_a.unstack()
        ll_s = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'AgeStd'),:].copy()
        ll_s_u = ll_s.unstack()
        ll_n_a_u= ll_n_u.join(ll_a_u)
        ll_n_a_s_u= ll_n_a_u.join(ll_s_u)
        ll_n_a_s_u.columns =['_'.join(col) for col in ll_n_a_s_u.columns.values]
        ll_n_a_s_u.reset_index(inplace=True)
        
        for demog_col in demog_col_List:
            origCol = demog_col
            
            col_N= '%s_N' %(origCol)
            col_Std = '%s_AgeStd' %(origCol)
            col_Age = '%s_MeanAge' %(origCol)
            col_t = '%s_t' %(origCol)
            ll_n_a_s_u[col_t] = ll_n_a_s_u.apply(lambda x : calculate_t(x[col_N], x[col_Std]), axis=1)
            
            #calculate pooled std
            std_pool_t =  ll_n_a_s_u[col_t].sum()
            std_pool_denom = ll_n_a_s_u[col_N].sum()-ll_n_a_s_u[col_N].count()
            std_pool = np.sqrt(std_pool_t/std_pool_denom)
            
            #calc pooled n and age
            N_2use = ll_n_a_s_u[col_N].sum()
            Age_2use = (ll_n_a_s_u[col_Age] * ll_n_a_s_u[col_N]).sum() /ll_n_a_s_u[col_N].sum() #weighted average
            
            Std_2use = std_pool
            
           
            
            
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','N'),demog_col]=N_2use
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','MeanAge'),demog_col]=Age_2use
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','AgeStd'),demog_col]=Std_2use
            
          


C:\Users\parfl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


In [70]:
llu = ll.unstack()      
llu.columns =['_'.join(col) for col in llu.columns.values]
llu.reset_index(inplace=True)

llu_copy = llu.copy()
demog_col_List =['All_mf', 'All_m', 'All_f', 'Q0_mf', 'Q0_m', 'Q0_f', 'Q2_mf', 'Q2_m', 'Q2_f', 'Q4_mf', 'Q4_m', 'Q4_f']
for demog_col in demog_col_List:
    origCol = demog_col

    col_N= '%s_N' %(origCol)
    col_Std = '%s_AgeStd' %(origCol)
    col_Age = '%s_MeanAge' %(origCol)
    col_AgeStd = '%s_Age(std)' %(origCol)
    llu_copy[col_AgeStd] = llu_copy.apply(lambda x : "{me:.2f} ({st:.2f})".format(me = x[col_Age], st=x[col_Std]), axis=1)
    llu_copy[col_N] = llu_copy[col_N].apply(lambda x: int(x))
    llu_copy.drop(labels = [col_Std, col_Age], axis=1, inplace=True)
    
llu_copy_g = llu_copy.groupby(['dis_n_denom','reg', 'year' ]).agg({'All_mf_N':'sum', 'All_m_N':'sum', 'All_f_N':'sum',
                                                                   'Q0_mf_N':'sum', 'Q0_m_N':'sum', 'Q0_f_N':'sum', 
                                                                   'Q2_mf_N':'sum', 'Q2_m_N':'sum', 'Q2_f_N':'sum',
                                                                   'Q4_mf_N':'sum', 'Q4_m_N':'sum', 'Q4_f_N':'sum',
                                                                   'All_mf_Age(std)':'first', 'All_m_Age(std)':'first', 'All_f_Age(std)':'first', 
                                                                   'Q0_mf_Age(std)':'first', 'Q0_m_Age(std)':'first', 'Q0_f_Age(std)':'first', 
                                                                   'Q2_mf_Age(std)':'first', 'Q2_m_Age(std)':'first', 'Q2_f_Age(std)':'first',
                                                                   'Q4_mf_Age(std)':'first', 'Q4_m_Age(std)':'first', 'Q4_f_Age(std)':'first'
                                                                  })

# reverse indeces to biggest first
llu_copy_g_r = llu_copy_g.reindex(index=llu_copy_g.index[::-1])
#select only interesing colummns
llu_copy_g_r_c = llu_copy_g_r[['All_mf_N',
 'All_mf_Age(std)',
 'All_m_N',
 'All_m_Age(std)',
 'All_f_N',
 'All_f_Age(std)',
 'Q0_mf_N',
 'Q0_mf_Age(std)',
 'Q0_m_N',
 'Q0_m_Age(std)',
 'Q0_f_N',
 'Q0_f_Age(std)',
 'Q2_mf_N',
 'Q2_mf_Age(std)',
 'Q2_m_N',
 'Q2_m_Age(std)',
 'Q2_f_N',
 'Q2_f_Age(std)',
 'Q4_mf_N',
 'Q4_mf_Age(std)',
 'Q4_m_N',
 'Q4_m_Age(std)',
 'Q4_f_N',
 'Q4_f_Age(std)']]

DisByRegByYear_temp = llu_copy_g_r_c.copy() #used for calculating rates
DisByRegByYear = DisByRegByYear_temp.copy()

excelName2save_df_denomsTable = "Table_Dis_numbers_by_register_year_feb23.xlsx" 
llu_copy_g_r_c.to_excel(excelName2save_df_denomsTable)

### Create index (row and column) to use for selecting subset of table, and save to excel

In [71]:
# select proper rows and columns
intColsSave = ['All_mf_N', 'All_mf_Age(std)',
        'Q0_mf_N', 'Q0_mf_Age(std)', 'Q4_mf_N','Q4_mf_Age(std)'] 

# create mulitiindex from cartesian merge to pressent rows in order
dis = ['All','Cardiovascular', 'Neoplasm','Neurological','Chronic_respiratory','Diabetes']
care= ['Death', 'SI_u', 'SO_u', 'PC_u']
years = ['FullPeriod', 2017,2011,2004]
CustomOrderedMultiIndex = pd.MultiIndex.from_product([dis, care, years], names = ["dis_n_denom", "reg","year"])


llu_copy_g_r_c_trimemd = llu_copy_g_r_c.loc[CustomOrderedMultiIndex,intColsSave]
excelName2save_df_denomsTable_trimmed = "Table_Dis_numbers_by_register_year_trimmed_feb23.xlsx" 
llu_copy_g_r_c_trimemd.to_excel(excelName2save_df_denomsTable_trimmed)

## Create demographic table of NPR of register (NPR vs PC), by year, group and by N and age

In [276]:
# if not building df_results from scratch, load here
df_denom = pd.read_excel("Demographic_by_registerpop_year_june23.xlsx" )
df_denom.drop(columns='Unnamed: 0', inplace=True) # clean
df_denom

,dis_n_denom,year,reg,measure,All_mf,All_m,All_f,Q0_mf,Q0_m,Q0_f,Q2_mf,Q2_m,Q2_f,Q4_mf,Q4_m,Q4_f
0,TotPop,2004,NPR_denom,N,7086141,3473650,3612491,1417981,695072,722909,1.417187e+06,694718.000000,722469.000000,1416834,694563,722271
1,TotPop,2004,NPR_denom,MeanAge,48.49037,47.317348,49.618309,48.488549,47.315823,49.616116,4.849095e+01,47.316278,49.620510,48.490403,47.317875,49.617951
2,TotPop,2004,NPR_denom,AgeStd,19.047432,18.342213,19.63578,19.048574,18.344038,19.636318,1.904782e+01,18.342688,19.635921,19.046066,18.341123,19.634268
3,TotPop,2004,NPR_denom,married,43.14%,43.93%,42.37%,37.01%,40.92%,33.25%,NaN,NaN,NaN,48.38%,44.65%,51.97%
4,TotPop,2004,NPR_denom,chi2_married,"χ²=37474.75, p<.001",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,TotPop,2017,PC_denom,married,40.77%,41.20%,40.34%,32.69%,35.31%,30.09%,NaN,NaN,NaN,46.49%,43.25%,49.69%
220,TotPop,2017,PC_denom,chi2_married,"χ²=57585.29, p<.001",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,TotPop,2017,PC_denom,SweBorn,80.71%,81.00%,80.42%,56.09%,55.53%,56.65%,NaN,NaN,NaN,90.78%,91.81%,89.77%
222,TotPop,2017,PC_denom,chi2_Swe,"χ²=447488.17, p<.001",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
reg_or_den_List = ['PC_u', 'SO_u', 'SI_u', 'Death', 'NPR_denom', 'PC_denom']
dis_n_denom_List = ['All', 'Cardiovascular', 'Neoplasm', 'Neurological','Diabetes','Chronic_respiratory', 'TotPop']
demog_col_List =["All_mf","All_m","All_f","Q0_mf","Q0_m","Q0_f","Q2_mf","Q2_m","Q2_f", "Q4_mf","Q4_m","Q4_f"]
dis_n_denom_List[-1:]

['TotPop']

In [280]:
reg_or_den_List = ['PC_u', 'SO_u', 'SI_u', 'Death', 'NPR_denom', 'PC_denom']
dis_n_denom_List = ['All', 'Cardiovascular', 'Neoplasm', 'Neurological','Diabetes','Chronic_respiratory', 'TotPop']
demog_col_List =["All_mf","All_m","All_f","Q0_mf","Q0_m","Q0_f","Q4_mf","Q4_m","Q4_f"]#"Q2_mf","Q2_m","Q2_f", 
ll = df_denom.groupby(['dis_n_denom','reg', 'year','measure' ]).first()
for nd, dis_or_tot in enumerate(dis_n_denom_List[-1:]):#, 'Cardiovascular']):#dis_or_tot_List[:-1]): 
    for dn, reg_or_den in enumerate(reg_or_den_List[-2:]) :
        
        ll_n = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'N'),:].copy()
        ll_n_u = ll_n.unstack()
        
        ll_a = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'MeanAge'),:].copy()
        ll_a_u = ll_a.unstack()
        
        ll_s = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'AgeStd'),:].copy()
        ll_s_u = ll_s.unstack()
        
        ll_sb = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'SweBorn'),:].copy()
        ll_sb_u = ll_sb.unstack()
        
        ll_m = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'married'),:].copy()
        ll_m_u = ll_m.unstack()
        
        ll_ink = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'ink_Q5_5_1byQ1_5_1'),:].copy()
        ll_ink_u = ll_ink.unstack()
        
        ll_chi2m = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'chi2_married'),:].copy()
        ll_chi2m_u = ll_chi2m.unstack()
        
        ll_chi2sb = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'chi2_Swe'),:].copy()
        ll_chi2sb_u = ll_chi2sb.unstack()
    
        
        
        ll_n_a_u= ll_n_u.join(ll_a_u)
        ll_n_a_s_u= ll_n_a_u.join(ll_s_u)
        
        ll_n_a_s_sb_u= ll_n_a_s_u.join(ll_sb_u)
        ll_n_a_s_sb_m_u= ll_n_a_s_sb_u.join(ll_m_u)
        ll_n_a_s_sb_m_ink_u= ll_n_a_s_sb_m_u.join(ll_ink_u)
        ll_n_a_s_sb_m_ink_chi2m_u= ll_n_a_s_sb_m_ink_u.join(ll_chi2m_u)
        ll_n_a_s_sb_m_ink_chi2m_chi2b_u= ll_n_a_s_sb_m_ink_chi2m_u.join(ll_chi2sb_u)
        
        ll_n_a_s_sb_m_ink_chi2m_chi2b_u.columns =['_'.join(col) for col in ll_n_a_s_sb_m_ink_chi2m_chi2b_u.columns.values]
        ll_n_a_s_sb_m_ink_chi2m_chi2b_u.reset_index(inplace=True)
        demo_tab = ll_n_a_s_sb_m_ink_chi2m_chi2b_u # shorter..
        
        # remove % sign, (only for All, Q0 and Q4 with non-nan)
        rateColumns = [element for element in list(demo_tab.columns) if
                       (element.endswith('married') or element.endswith('SweBorn')) and not element.endswith('chi2_married') and
                       (element.startswith('All') or element.startswith('Q0') or element.startswith('Q4'))]
        for rateCol in rateColumns: 
            demo_tab[rateCol] = demo_tab[rateCol].str[:-1]
            demo_tab[rateCol] = demo_tab[rateCol].astype('float32')
        
        #weighted average
        


        

### For tot population

In [308]:
       
reg_or_den_List = ['PC_u', 'SO_u', 'SI_u', 'Death', 'NPR_denom', 'PC_denom']
dis_n_denom_List = ['All', 'Cardiovascular', 'Neoplasm', 'Neurological','Diabetes','Chronic_respiratory', 'TotPop']
demog_col_List =["All_mf","All_m","All_f","Q0_mf","Q0_m","Q0_f","Q4_mf","Q4_m","Q4_f"]#"Q2_mf","Q2_m","Q2_f", 
ll = df_denom.groupby(['dis_n_denom','reg', 'year','measure' ]).first()
for nd, dis_or_tot in enumerate(dis_n_denom_List[-1:]):#, 'Cardiovascular']):#dis_or_tot_List[:-1]): 
    for dn, reg_or_den in enumerate(reg_or_den_List[-2:]) :
        
        ll_n = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'N'),:].copy()
        ll_n_u = ll_n.unstack()
        
        ll_a = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'MeanAge'),:].copy()
        ll_a_u = ll_a.unstack()
        
        ll_s = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'AgeStd'),:].copy()
        ll_s_u = ll_s.unstack()
        
        ll_sb = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'SweBorn'),:].copy()
        ll_sb_u = ll_sb.unstack()
        
        ll_m = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'married'),:].copy()
        ll_m_u = ll_m.unstack()
        
        ll_ink = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'ink_Q5_5_1byQ1_5_1'),:].copy()
        ll_ink_u = ll_ink.unstack()
        
        ll_chi2m = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'chi2_married'),:].copy()
        ll_chi2m_u = ll_chi2m.unstack()
        
        ll_chi2sb = ll.loc[(dis_or_tot, reg_or_den,slice(None), 'chi2_Swe'),:].copy()
        ll_chi2sb_u = ll_chi2sb.unstack()
    
        
        
        ll_n_a_u= ll_n_u.join(ll_a_u)
        ll_n_a_s_u= ll_n_a_u.join(ll_s_u)
        
        ll_n_a_s_sb_u= ll_n_a_s_u.join(ll_sb_u)
        ll_n_a_s_sb_m_u= ll_n_a_s_sb_u.join(ll_m_u)
        ll_n_a_s_sb_m_ink_u= ll_n_a_s_sb_m_u.join(ll_ink_u)
        ll_n_a_s_sb_m_ink_chi2m_u= ll_n_a_s_sb_m_ink_u.join(ll_chi2m_u)
        ll_n_a_s_sb_m_ink_chi2m_chi2b_u= ll_n_a_s_sb_m_ink_chi2m_u.join(ll_chi2sb_u)
        
        ll_n_a_s_sb_m_ink_chi2m_chi2b_u.columns =['_'.join(col) for col in ll_n_a_s_sb_m_ink_chi2m_chi2b_u.columns.values]
        ll_n_a_s_sb_m_ink_chi2m_chi2b_u.reset_index(inplace=True)
        demo_tab = ll_n_a_s_sb_m_ink_chi2m_chi2b_u # shorter..
        
        # remove % sign, (only for All, Q0 and Q4 with non-nan)
        rateColumns = [element for element in list(demo_tab.columns) if
                       (element.endswith('married') or element.endswith('SweBorn')) and not element.endswith('chi2_married') and
                       (element.startswith('All') or element.startswith('Q0') or element.startswith('Q4'))]
        for rateCol in rateColumns: 
            demo_tab[rateCol] = demo_tab[rateCol].str[:-1]
            demo_tab[rateCol] = demo_tab[rateCol].astype('float32')        
        
        
        
        for demog_col in demog_col_List:
            origCol = demog_col
            
            col_N= '%s_N' %(origCol)
            col_Std = '%s_AgeStd' %(origCol)
            col_Age = '%s_MeanAge' %(origCol)
            col_t = '%s_t' %(origCol)
            col_SweBorn = '%s_SweBorn' %(origCol)
            col_married = '%s_married' %(origCol)
            
            demo_tab[col_t] = demo_tab.apply(lambda x : calculate_t(x[col_N], x[col_Std]), axis=1)
            
            #calculate pooled std
            std_pool_t =  demo_tab[col_t].sum()
            std_pool_denom = demo_tab[col_N].sum()-demo_tab[col_N].count()
            std_pool = np.sqrt(std_pool_t/std_pool_denom)
            
            #calc pooled n and age
            N_2use = demo_tab[col_N].sum()
            Age_2use = (demo_tab[col_Age] * demo_tab[col_N]).sum() /demo_tab[col_N].sum() #weighted average
            
            Std_2use = std_pool
            
            
            #calc pooled ink_Q5_5_1byQ1_5_1
            if origCol in ["All_mf"]: #All_mf_ink_Q5_5_1byQ1_5_1 is only in All_mf
                demo_tab['All_mf_ink_Q5_5_1byQ1_5_1'] = demo_tab["All_mf_ink_Q5_5_1byQ1_5_1"].astype('float32')
                ink_Q5_5_1byQ1_5_1_2use = (demo_tab['All_mf_ink_Q5_5_1byQ1_5_1'] * demo_tab['All_mf_N']).sum() /demo_tab['All_mf_N'].sum() #weighted eman
                ink_Q5_5_1byQ1_5_1_2use = f"{ink_Q5_5_1byQ1_5_1_2use:.2f}"
                
            #calc pooled FullPeriodSweBorn
            R_SweBorn_2use = (demo_tab[col_SweBorn] * demo_tab[col_N]).sum() /demo_tab[col_N].sum() #weighted average
            R_SweBorn_2use = f"{R_SweBorn_2use:.2f}%"
            
            #calc pooled married
            R_married_2use = (demo_tab[col_married] * demo_tab[col_N]).sum() /demo_tab[col_N].sum() #weighted average
            R_married_2use = f"{R_married_2use:.2f}%"
            
           
            
            #specify the Fullperiod numbers 
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','N'),demog_col]=N_2use
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','MeanAge'),demog_col]=Age_2use
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','AgeStd'),demog_col]=Std_2use
            
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','SweBorn'),demog_col]=R_SweBorn_2use
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','married'),demog_col]=R_married_2use
            ll.loc[(dis_or_tot,reg_or_den,'FullPeriod','ink_Q5_5_1byQ1_5_1'),'All_mf']=ink_Q5_5_1byQ1_5_1_2use
          


In [309]:
llu = ll.unstack()      
llu.columns =['_'.join(col) for col in llu.columns.values]
llu.reset_index(inplace=True)

In [328]:
llu_copy = llu.copy()
llu_copy = llu_copy.dropna(axis=1, how='all')

demog_col_List =['All_mf', 'All_m', 'All_f', 'Q0_mf', 'Q0_m', 'Q0_f', 'Q4_mf', 'Q4_m', 'Q4_f'] #['All_mf', 'All_m', 'All_f', 'Q0_mf', 'Q0_m', 'Q0_f', 'Q2_mf', 'Q2_m', 'Q2_f', 'Q4_mf', 'Q4_m', 'Q4_f']
for demog_col in demog_col_List:
    origCol = demog_col


    col_N= '%s_N' %(origCol)
    col_Std = '%s_AgeStd' %(origCol)
    col_Age = '%s_MeanAge' %(origCol)
    col_AgeStd = '%s_Age(std)' %(origCol)
    col_t = '%s_t' %(origCol)
    col_SweBorn = '%s_SweBorn' %(origCol)
    col_married = '%s_married' %(origCol)
    
    
    llu_copy[col_AgeStd] = llu_copy.apply(lambda x : "{me:.2f} ({st:.2f})".format(me = x[col_Age], st=x[col_Std]), axis=1)
    llu_copy[col_N] = llu_copy[col_N].apply(lambda x: int(x))
    llu_copy.drop(labels = [col_Std, col_Age], axis=1, inplace=True)
    
llu_copy_g = llu_copy.groupby(['dis_n_denom','reg', 'year' ]).agg({'All_mf_N':'sum', 'All_m_N':'sum', 'All_f_N':'sum',
                                                                   'Q0_mf_N':'sum', 'Q0_m_N':'sum', 'Q0_f_N':'sum', 
                                                                   #'Q2_mf_N':'sum', 'Q2_m_N':'sum', 'Q2_f_N':'sum',
                                                                   'Q4_mf_N':'sum', 'Q4_m_N':'sum', 'Q4_f_N':'sum',
                                                                   'All_mf_Age(std)':'first', 'All_m_Age(std)':'first', 'All_f_Age(std)':'first', 
                                                                   'Q0_mf_Age(std)':'first',  'Q0_m_Age(std)':'first',  'Q0_f_Age(std)':'first', 
                                                                   #'Q2_mf_Age(std)':'first',  'Q2_m_Age(std)':'first',  'Q2_f_Age(std)':'first',
                                                                   'Q4_mf_Age(std)':'first',  'Q4_m_Age(std)':'first',  'Q4_f_Age(std)':'first',
                                                                   'All_mf_SweBorn':'first',  'All_m_SweBorn':'first',  'All_f_SweBorn':'first',
                                                                   'Q0_mf_SweBorn':'first',   'Q0_m_SweBorn':'first',   'Q0_f_SweBorn':'first',
                                                                   #'Q2_mf_SweBorn':'first',   'Q2_m_SweBorn':'first',   'Q2_f_SweBorn':'first',
                                                                   'Q4_mf_SweBorn':'first',   'Q4_m_SweBorn':'first',   'Q4_f_SweBorn':'first',
                                                                   'All_mf_married':'first',  'All_m_married':'first',  'All_f_married':'first',
                                                                   'Q0_mf_married':'first',   'Q0_m_married':'first',   'Q0_f_married':'first',
                                                                   #'Q2_mf_married':'first',   'Q2_m_married':'first',   'Q2_f_married':'first',
                                                                   'Q4_mf_married':'first',   'Q4_m_married':'first',   'Q4_f_married':'first',
                                                                   'All_mf_ink_Q5_5_1byQ1_5_1':'first',
                                                                   'All_mf_chi2_Swe':'first',
                                                                   'All_mf_chi2_married':'first'
                                                                  })

# reverse indeces to biggest first
llu_copy_g_r = llu_copy_g.reindex(index=llu_copy_g.index[::-1])
#select only interesing colummns
llu_copy_g_r_c = llu_copy_g_r[['All_mf_N',
 'All_mf_Age(std)',
 'All_m_N',
 'All_mf_chi2_Swe', 
 'All_mf_chi2_married',
 'All_mf_ink_Q5_5_1byQ1_5_1',
 'All_mf_married',
 'All_mf_SweBorn',                              
 'All_m_Age(std)',
 'All_f_N',
 'All_f_Age(std)',
 'Q0_mf_N',
 'Q0_mf_Age(std)',
 'Q0_mf_SweBorn',
 'Q0_mf_married',                             
 'Q0_m_N',
 'Q0_m_Age(std)',
 'Q0_f_N',
 'Q0_f_Age(std)',
 #'Q2_mf_N',
 # 'Q2_mf_Age(std)',
 # 'Q2_m_N',
 # 'Q2_m_Age(std)',
 # 'Q2_f_N',
 # 'Q2_f_Age(std)',
 'Q4_mf_N',
 'Q4_mf_Age(std)',
 'Q4_mf_SweBorn',
 'Q4_mf_married',
 'Q4_m_N',
 'Q4_m_Age(std)',
 'Q4_f_N',
 'Q4_f_Age(std)']]

PopByRegByYear_temp = llu_copy_g_r_c.copy()
PopByRegByYear = PopByRegByYear_temp.copy() #used for calculating rates

intColsSave = ['All_mf_N', 'All_mf_Age(std)',
        'Q0_mf_N', 'Q0_mf_Age(std)', 'Q4_mf_N','Q4_mf_Age(std)'] 
# new order
intColsSave = ['All_mf_N', 'Q0_mf_N','Q4_mf_N',
               'All_mf_Age(std)','Q0_mf_Age(std)','Q4_mf_Age(std)',
               'All_mf_SweBorn', 'Q0_mf_SweBorn','Q4_mf_SweBorn','All_mf_chi2_Swe',
               'All_mf_married', 'Q0_mf_married','Q4_mf_married','All_mf_chi2_married',
               'All_mf_ink_Q5_5_1byQ1_5_1']


PopByRegByYearNew_trimmed = PopByRegByYear.loc[(slice(None), slice(None), ('FullPeriod', 2017,2011,2004)), intColsSave]
PopByRegByYearNew_trimmed = PopByRegByYearNew_trimmed.sort_index(ascending=[True,True,False])

excelName2save_df_denomsTable = "Table_popdenom_by_register_year_trimmed_june23.xlsx" 
PopByRegByYearNew_trimmed.to_excel(excelName2save_df_denomsTable)


# Create tabel of Disease by register by year RATES

In [76]:
# when running in second time, load data
# load demographic
PopByRegByYear = pd.read_excel('Demographic_by_registerpop_year_feb23.xlsx')
DisByRegByYearNew = pd.read_excel('Table_popdenom_by_register_year_trimmed_feb23.xlsx')


In [77]:

DisByRegByYear = DisByRegByYear_temp.copy()
PopByRegByYear = PopByRegByYear_temp.copy()

In [83]:
# create rates, cell wise operation
# // FullPeriod for chronic resp and Diabetes didn't work, for some reason??
DisByRegByYearNew = DisByRegByYear.copy()    
dfNprPop = PopByRegByYear.loc[(slice(None),'NPR_denom'),:].copy()
dfPcPop = PopByRegByYear.loc[(slice(None),'PC_denom'),:].copy()
# Columns to calculate reate for
intCols = ['All_mf_N', 'All_m_N',  'All_f_N',
        'Q0_mf_N',  'Q0_m_N',
       'Q0_f_N', 'Q2_mf_N', 'Q2_m_N',  'Q2_f_N',  'Q4_mf_N',
        'Q4_m_N', 'Q4_f_N'] 

for dis in [ 'All','Cardiovascular', 'Neoplasm','Neurological','Chronic_respiratory','Diabetes']:
    for reg in ['Death',  'SI_u', 'SO_u', 'PC_u']:
        for years in [2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017, 'FullPeriod']:
            if reg == 'PC_u':
                denomDf =dfPcPop 
            else:
                denomDf =dfNprPop

            for nom_col in intCols: #loop over all int cols
                #if not (((dis == "Chronic_respiratory") or (dis == "Diabetes"))  & (years=='FullPeriod')):
                DisByRegByYearNew.loc[(dis, reg,years), nom_col] = round(int(DisByRegByYearNew.loc[(dis, reg,years), nom_col]) / int(denomDf.loc[(slice(None), slice(None),years),nom_col])*100000)
    
   

C:\Users\parfl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


In [85]:
# select proper rows and columns
intColsSave = ['All_mf_N', 'All_mf_Age(std)',
        'Q0_mf_N', 'Q0_mf_Age(std)', 'Q4_mf_N','Q4_mf_Age(std)'] 

# create mulitiindex from cartesian merge to pressent rows in order
dis = ['All','Cardiovascular', 'Neoplasm','Neurological','Chronic_respiratory','Diabetes']
care= ['Death', 'SI_u', 'SO_u', 'PC_u']
years = ['FullPeriod', 2017,2011,2004]

CustomOrderedMultiIndex = pd.MultiIndex.from_product([dis, care, years], names = ["dis_n_denom", "reg","year"])

In [87]:
DisByRegByYearNewc_trimemd = DisByRegByYearNew.loc[CustomOrderedMultiIndex,intColsSave]
excelName2save_df_denomsTable_trimmed = "Table_Dis_rates_by_register_year_trimmed_feb23.xlsx" 
DisByRegByYearNewc_trimemd.to_excel(excelName2save_df_denomsTable_trimmed)

In [ ]:
DisByRegByYearNew_trimmed = DisByRegByYearNew.loc[(slice(None), slice(None), ('FullPeriod', 2017,2011,2004)), intColsSave]
DisByRegByYearNew_trimmed = DisByRegByYearNew_trimmed.sort_index(ascending=[True,True,False])

excelName2save_df_denomsTable = "Table_DiseaseRates_by_register_year_jan23_Rates.xlsx" 
DisByRegByYearNew_trimmed.to_excel(excelName2save_df_denomsTable)

NameError: name 'DisByRegByYearNew' is not defined

In [24]:
excelName2save_df_denomsTable = "Table_DiseaseRates_by_register_year_jan23_Rates.xlsx" 
ff=pd.read_excel(excelName2save_df_denomsTable)

ff

,dis_n_denom,reg,year,All_mf_N,All_mf_Age(std),Q0_mf_N,Q0_mf_Age(std),Q4_mf_N,Q4_mf_Age(std)
0,TotPop,NPR_denom,FullPeriod,105198091,48.49 (19.42),21041685,48.49 (19.42),21037421,48.49 (19.42)
1,NaN,NaN,2017,7994344,48.74 (19.63),1598945,48.74 (19.63),1598739,48.74 (19.63)
2,NaN,NaN,2011,7612768,48.12 (19.62),1522709,48.12 (19.62),1522410,48.12 (19.62)
3,NaN,NaN,2004,7076444,48.49 (19.05),1415475,48.49 (19.05),1415087,48.49 (19.05)
4,NaN,PC_denom,FullPeriod,77703198,48.16 (19.36),15592579,47.94 (19.20),16364837,48.28 (19.35)
5,NaN,NaN,2017,7136015,48.63 (19.59),1436372,48.64 (19.56),1446094,48.61 (19.58)
6,NaN,NaN,2011,5831702,47.83 (19.53),1175666,47.66 (19.41),1218986,47.91 (19.53)
7,NaN,NaN,2004,4095167,47.84 (18.91),813256,47.16 (18.48),922039,48.18 (18.91)
